[View in Colaboratory](https://colab.research.google.com/github/sohamshashank1/MLtoolkit/blob/master/TF_Skip_gram_&_CBOW.ipynb)

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import string
import requests
import collections
import io
import tarfile
import urllib.request
# from nltk.corpus import stopwords
sess = tf.Session()
batch_size = 50
embedding_size = 200
vocabulary_size = 10000
generations = 50000
print_loss_every = 500
num_sampled = int(batch_size/2)
window_size = 2
# stops = stopwords.words('english')
print_valid_every = 2000
valid_words = ['cliche', 'love', 'hate', 'silly', 'sad']

In [0]:
def load_movie_data():
    save_folder_name = 'temp'
    pos_file = os.path.join(save_folder_name, 'rt-polaritydata', 'rt-polarity.pos')
    neg_file = os.path.join(save_folder_name, 'rt-polaritydata', 'rt-polarity.neg')

    # Check if files are already downloaded
    if not os.path.exists(os.path.join(save_folder_name, 'rt-polaritydata')):
        movie_data_url = 'http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz'

        # Save tar.gz file
        req = requests.get(movie_data_url, stream=True)
        with open('temp_movie_review_temp.tar.gz', 'wb') as f:
            for chunk in req.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    f.flush()
        # Extract tar.gz file into temp folder
        tar = tarfile.open('temp_movie_review_temp.tar.gz', "r:gz")
        tar.extractall(path='temp')
        tar.close()

    pos_data = []
    with open(pos_file, 'r', encoding='latin-1') as f:
        for line in f:
            pos_data.append(line.encode('ascii', errors='ignore').decode())
    f.close()
    pos_data = [x.rstrip() for x in pos_data]

    neg_data = []
    with open(neg_file, 'r', encoding='latin-1') as f:
        for line in f:
            neg_data.append(line.encode('ascii', errors='ignore').decode())
    f.close()
    neg_data = [x.rstrip() for x in neg_data]
    
    texts = pos_data + neg_data
    target = [1] * len(pos_data) + [0] * len(neg_data)
    
    return texts, target

texts, target = load_movie_data()

In [5]:
texts.head()

['the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .',
 'the gorgeously elaborate continuation of " the lord of the rings " trilogy is so huge that a column of words cannot adequately describe co-writer/director peter jackson\'s expanded vision of j . r . r . tolkien\'s middle-earth .',
 'effective but too-tepid biopic',
 'if you sometimes like to go to the movies to have fun , wasabi is a good place to start .',
 "emerges as something rare , an issue movie that's so honest and keenly observed that it doesn't feel like one .",
 'the film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game .',
 'offers that rare combination of entertainment and education .',
 'perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions .',
 "steers turns i

In [0]:
def normalize_text(texts):
    # Lower case
    texts = [x.lower() for x in texts]
    # Remove punctuation
    texts = [''.join(c for c in x if c not in string.punctuation) for x in texts]
    # Remove numbers
    texts = [''.join(c for c in x if c not in '0123456789') for x in texts]
    # Remove stopwords
#     texts = [' '.join([word for word in x.split() if word not in (stops)]) for x in texts]
    # Trim extra whitespace
    texts = [' '.join(x.split()) for x in texts]
    
    return(texts)
texts = normalize_text(texts)

In [14]:
texts

['the rock is destined to be the st centurys new conan and that hes going to make a splash even greater than arnold schwarzenegger jeanclaud van damme or steven segal',
 'the gorgeously elaborate continuation of the lord of the rings trilogy is so huge that a column of words cannot adequately describe cowriterdirector peter jacksons expanded vision of j r r tolkiens middleearth',
 'effective but tootepid biopic',
 'if you sometimes like to go to the movies to have fun wasabi is a good place to start',
 'emerges as something rare an issue movie thats so honest and keenly observed that it doesnt feel like one',
 'the film provides some great insight into the neurotic mindset of all comics even those who have reached the absolute top of the game',
 'offers that rare combination of entertainment and education',
 'perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions',
 'steers turns in a snappy screenplay that curls at the edges its so c

**CODE STARTS HERE**

In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import string
import requests
import collections
import io
import gzip
import tarfile
import urllib.request
# from nltk.corpus import stopwords
from tensorflow.python.framework import ops
ops.reset_default_graph()

# os.chdir(os.path.dirname(os.path.realpath(__file__)))

# Start a graph session
sess = tf.Session()

# Declare model parameters
batch_size = 100
embedding_size = 200
vocabulary_size = 10000
generations = 100000
print_loss_every = 2000

num_sampled = int(batch_size/2)    # Number of negative examples to sample.
window_size = 2       # How many words to consider left and right.

# Declare stop words
# stops = stopwords.words('english')

# We pick five test words. We are expecting synonyms to appear
print_valid_every = 5000
valid_words = ['cliche', 'love', 'hate', 'silly', 'sad']
# Later we will have to transform these into indices


# Load the movie review data
# Check if data was downloaded, otherwise download it and save for future use
def load_movie_data():
    save_folder_name = 'temp'
    pos_file = os.path.join(save_folder_name, 'rt-polaritydata', 'rt-polarity.pos')
    neg_file = os.path.join(save_folder_name, 'rt-polaritydata', 'rt-polarity.neg')

    # Check if files are already downloaded
    if not os.path.exists(os.path.join(save_folder_name, 'rt-polaritydata')):
        movie_data_url = 'http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz'

        # Save tar.gz file
        req = requests.get(movie_data_url, stream=True)
        with open('temp_movie_review_temp.tar.gz', 'wb') as f:
            for chunk in req.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    f.flush()
        # Extract tar.gz file into temp folder
        tar = tarfile.open('temp_movie_review_temp.tar.gz', "r:gz")
        tar.extractall(path='temp')
        tar.close()

    pos_data = []
    with open(pos_file, 'r', encoding='latin-1') as f:
        for line in f:
            pos_data.append(line.encode('ascii', errors='ignore').decode())
    f.close()
    pos_data = [x.rstrip() for x in pos_data]

    neg_data = []
    with open(neg_file, 'r', encoding='latin-1') as f:
        for line in f:
            neg_data.append(line.encode('ascii', errors='ignore').decode())
    f.close()
    neg_data = [x.rstrip() for x in neg_data]
    
    texts = pos_data + neg_data
    target = [1] * len(pos_data) + [0] * len(neg_data)
    
    return texts, target

texts, target = load_movie_data()


# Normalize text
def normalize_text(texts):
    # Lower case
    texts = [x.lower() for x in texts]

    # Remove punctuation
    texts = [''.join(c for c in x if c not in string.punctuation) for x in texts]

    # Remove numbers
    texts = [''.join(c for c in x if c not in '0123456789') for x in texts]

    # Remove stopwords
#     texts = [' '.join([word for word in x.split() if word not in stops]) for x in texts]

    # Trim extra whitespace
    texts = [' '.join(x.split()) for x in texts]
    
    return texts
    
texts = normalize_text(texts)

# Texts must contain at least 3 words
target = [target[ix] for ix, x in enumerate(texts) if len(x.split()) > 2]
texts = [x for x in texts if len(x.split()) > 2]



In [28]:
# Build dictionary of words
def build_dictionary(sentences, vocabulary_size):
    # Turn sentences (list of strings) into lists of words
    split_sentences = [s.split() for s in sentences]
    words = [x for sublist in split_sentences for x in sublist]
    
    # Initialize list of [word, word_count] for each word, starting with unknown
    count = [['RARE', -1]]
    
    # Now add most frequent words, limited to the N-most frequent (N=vocabulary size)
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    
    # Now create the dictionary
    word_dict = {}
    # For each word, that we want in the dictionary, add it, then make it
    # the value of the prior dictionary length
    for word, word_count in count:
        word_dict[word] = len(word_dict)
    
    return word_dict
word_dictionary = build_dictionary(texts, vocabulary_size)
word_dictionary_rev = dict(zip(word_dictionary.values(), word_dictionary.keys()))
 
word_dictionary

{'RARE': 0,
 'the': 1,
 'a': 2,
 'and': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'in': 7,
 'that': 8,
 'its': 9,
 'it': 10,
 'as': 11,
 'but': 12,
 'with': 13,
 'film': 14,
 'this': 15,
 'for': 16,
 'an': 17,
 'movie': 18,
 'be': 19,
 'on': 20,
 'you': 21,
 'not': 22,
 'by': 23,
 'about': 24,
 'one': 25,
 'more': 26,
 'like': 27,
 'has': 28,
 'are': 29,
 'at': 30,
 'from': 31,
 'than': 32,
 'all': 33,
 'his': 34,
 'have': 35,
 'so': 36,
 'if': 37,
 'or': 38,
 'story': 39,
 'i': 40,
 'too': 41,
 'just': 42,
 'who': 43,
 'what': 44,
 'into': 45,
 'most': 46,
 'out': 47,
 'no': 48,
 'much': 49,
 'even': 50,
 'good': 51,
 'up': 52,
 'will': 53,
 'comedy': 54,
 'time': 55,
 'characters': 56,
 'can': 57,
 'some': 58,
 'films': 59,
 'only': 60,
 'little': 61,
 'way': 62,
 'their': 63,
 'funny': 64,
 'make': 65,
 'movies': 66,
 'enough': 67,
 'been': 68,
 'very': 69,
 'your': 70,
 'never': 71,
 'when': 72,
 'makes': 73,
 'there': 74,
 'may': 75,
 'which': 76,
 'us': 77,
 'best': 78,
 'work': 79,
 'di

In [31]:
def text_to_numbers(sentences, word_dict):
    # Initialize the returned data
    data = []
    for sentence in sentences:
        sentence_data = []
        # For each word, either use selected index or rare word index
        for word in sentence.split(' '):
            if word in word_dict:
                word_ix = word_dict[word]
            else:
                word_ix = 0
            sentence_data.append(word_ix)
        data.append(sentence_data)
    return data

text_data = text_to_numbers(texts, word_dictionary)
text_data

[[1,
  642,
  6,
  2525,
  5,
  19,
  1,
  2275,
  0,
  94,
  7070,
  3,
  8,
  313,
  242,
  5,
  65,
  2,
  3759,
  50,
  2828,
  32,
  1510,
  2276,
  0,
  1628,
  7071,
  38,
  871,
  0],
 [1,
  3224,
  2061,
  7072,
  4,
  1,
  4463,
  4,
  1,
  2829,
  4464,
  6,
  36,
  1059,
  8,
  2,
  7073,
  4,
  768,
  830,
  4465,
  2526,
  7074,
  1060,
  5430,
  4466,
  643,
  4,
  1234,
  2830,
  2830,
  7075,
  0],
 [672, 12, 0, 1895],
 [37, 21, 246, 27, 5, 191, 5, 1, 66, 5, 35, 128, 7076, 6, 2, 51, 309, 5, 644],
 [1314,
  11,
  98,
  377,
  17,
  1752,
  18,
  108,
  36,
  505,
  3,
  5431,
  3760,
  8,
  10,
  83,
  162,
  27,
  25],
 [1,
  14,
  673,
  58,
  129,
  597,
  45,
  1,
  4467,
  3761,
  4,
  33,
  3762,
  50,
  110,
  43,
  35,
  4468,
  1,
  3225,
  1235,
  4,
  1,
  537],
 [281, 8, 377, 1171, 4, 251, 3, 3226],
 [598,
  48,
  163,
  138,
  101,
  28,
  26,
  1629,
  0,
  8,
  1,
  872,
  5,
  697,
  6,
  7077,
  13,
  51,
  1315],
 [2831,
  329,
  7,
  2,
  2832,
  316,

In [0]:
valid_examples = [word_dictionary[x] for x in valid_words]


# Generate data randomly (N words behind, target, N words ahead)
def generate_batch_data(sentences, batch_size, window_size, method='skip_gram'):
    # Fill up data batch
    batch_data = []
    label_data = []
    while len(batch_data) < batch_size:
        # select random sentence to start
        rand_sentence = np.random.choice(sentences)
        # Generate consecutive windows to look at
        window_sequences = [rand_sentence[max((ix - window_size), 0):(ix + window_size + 1)] for ix, x in enumerate(rand_sentence)]
        # Denote which element of each window is the center word of interest
        label_indices = [ix if ix < window_size else window_size for ix, x in enumerate(window_sequences)]
        
        # Pull out center word of interest for each window and create a tuple for each window
        if method == 'skip_gram':
            batch_and_labels = [(x[y], x[:y] + x[(y+1):]) for x, y in zip(window_sequences, label_indices)]
            # Make it in to a big list of tuples (target word, surrounding word)
            tuple_data = [(x, y_) for x, y in batch_and_labels for y_ in y]
        elif method == 'cbow':
            batch_and_labels = [(x[:y] + x[(y + 1):], x[y]) for x, y in zip(window_sequences, label_indices)]
            # Make it in to a big list of tuples (target word, surrounding word)
            tuple_data = [(x_, y) for x, y in batch_and_labels for x_ in x]
        else:
            raise ValueError('Method {} not implemented yet.'.format(method))
            
        # extract batch and labels
        batch, labels = [list(x) for x in zip(*tuple_data)]
        batch_data.extend(batch[:batch_size])
        label_data.extend(labels[:batch_size])
    # Trim batch and label at the end
    batch_data = batch_data[:batch_size]
    label_data = label_data[:batch_size]
    
    # Convert to numpy array
    batch_data = np.array(batch_data)
    label_data = np.transpose(np.array([label_data]))
    
    return batch_data, label_data


# Define Embeddings:
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))

# NCE loss parameters
nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                                              stddev=1.0 / np.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Create data/target placeholders
x_inputs = tf.placeholder(tf.int32, shape=[batch_size])
y_target = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

# Lookup the word embedding:
embed = tf.nn.embedding_lookup(embeddings, x_inputs)

# Get loss from prediction
loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                     biases=nce_biases,
                                     labels=y_target,
                                     inputs=embed,
                                     num_sampled=num_sampled,
                                     num_classes=vocabulary_size))
                                     
# Create optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1.0).minimize(loss)

# Cosine similarity between words
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

# Add variable initializer.
init = tf.global_variables_initializer()
sess.run(init)

# Run the skip gram model.
loss_vec = []
loss_x_vec = []
for i in range(generations):
    batch_inputs, batch_labels = generate_batch_data(text_data, batch_size, window_size)
    feed_dict = {x_inputs: batch_inputs, y_target: batch_labels}

    # Run the train step
    sess.run(optimizer, feed_dict=feed_dict)

    # Return the loss
    if (i + 1) % print_loss_every == 0:
        loss_val = sess.run(loss, feed_dict=feed_dict)
        loss_vec.append(loss_val)
        loss_x_vec.append(i+1)
        print('Loss at step {} : {}'.format(i+1, loss_val))
      
    # Validation: Print some random words and top 5 related words
    if (i+1) % print_valid_every == 0:
        sim = sess.run(similarity, feed_dict=feed_dict)
        for j in range(len(valid_words)):
            valid_word = valid_words[j]
            # top_k = number of nearest neighbors
            top_k = 5
            nearest = (-sim[j, :]).argsort()[1:top_k+1]
            log_str = "Nearest to {}:".format(valid_word)
            for k in range(top_k):
                close_word = word_dictionary_rev[nearest[k]]
                log_str = '{} {},'.format(log_str, close_word)
            print(log_str)

Loss at step 2000 : 5.873992919921875
Loss at step 4000 : 12.405755996704102
Nearest to cliche: masterfully, not, catches, las, oppressively,
Nearest to love: jagloms, ghosts, waves, sizzle, type,
Nearest to hate: seen, buy, close, grant, taken,
Nearest to silly: ha, convey, sublime, opinion, flavorful,
Nearest to sad: jagloms, stores, RARE, ignites, musicals,
Loss at step 6000 : 4.578869819641113
Loss at step 8000 : 4.959242820739746
Loss at step 10000 : 4.2763447761535645
Nearest to cliche: predict, masterfully, not, catches, fart,
Nearest to love: jagloms, waves, ghosts, flotsam, emphasizing,
Nearest to hate: seen, buy, explored, that, about,
Nearest to silly: ha, opinion, convey, sublime, empire,
Nearest to sad: jagloms, stores, song, RARE, ignites,
Loss at step 12000 : 4.503476619720459
Loss at step 14000 : 4.029344081878662
Nearest to cliche: predict, masterfully, catches, not, fart,
Nearest to love: jagloms, waves, ghosts, flotsam, sizzle,
Nearest to hate: seen, buy, explored, a

In [0]:
print('Starting Training')
loss_vec = []
loss_x_vec = []
for i in range(generations):
    batch_inputs, batch_labels = generate_batch_data(text_data, batch_size,
                                                                  window_size, method='cbow')
    feed_dict = {x_inputs: batch_inputs, y_target: batch_labels}

    # Run the train step
    sess.run(optimizer, feed_dict=feed_dict)

    # Return the loss
    if (i+1) % print_loss_every == 0:
        loss_val = sess.run(loss, feed_dict=feed_dict)
        loss_vec.append(loss_val)
        loss_x_vec.append(i+1)
        print('Loss at step {} : {}'.format(i+1, loss_val))
      
    # Validation: Print some random words and top 5 related words
    if (i+1) % print_valid_every == 0:
        sim = sess.run(similarity, feed_dict=feed_dict)
        for j in range(len(valid_words)):
            valid_word = word_dictionary_rev[valid_examples[j]]
            top_k = 5  # number of nearest neighbors
            nearest = (-sim[j, :]).argsort()[1:top_k+1]
            log_str = "Nearest to {}:".format(valid_word)
            for k in range(top_k):
                close_word = word_dictionary_rev[nearest[k]]
                log_str = '{} {},' .format(log_str, close_word)
            print(log_str)
            

Starting Training
Loss at step 2000 : 3.4424338340759277
Loss at step 4000 : 3.1545839309692383
Nearest to cliche: masterfully, predict, satisfied, fart, las,
Nearest to love: premises, flotsam, jagloms, type, conventions,
Nearest to hate: explored, seen, feel, buy, limitations,
Nearest to silly: convictions, cheaplooking, trimming, empire, arnie,
Nearest to sad: flatter, chinas, rancid, stores, convincingly,
Loss at step 6000 : 3.672329902648926
Loss at step 8000 : 3.0770370960235596
Loss at step 10000 : 3.698244333267212
Nearest to cliche: masterfully, predict, satisfied, las, fart,
Nearest to love: flotsam, premises, jagloms, type, conventions,
Nearest to hate: explored, seen, feel, buy, limitations,
Nearest to silly: convictions, cheaplooking, empire, rating, arnie,
Nearest to sad: flatter, chinas, rancid, stores, convincingly,
Loss at step 12000 : 3.362532615661621
Loss at step 14000 : 3.4934685230255127
Nearest to cliche: masterfully, predict, satisfied, las, fart,
Nearest to lov